# toi.jlの扱い

## 準備

In [9]:
using ITensors
using Quantics
function createTensors(N, D, d)
    tensors = Array{Float64,4}[]
    for i in 1:N
        if i == 1
            push!(tensors, rand(1, d, d,  D))
        elseif i == N
            push!(tensors, rand(D, d, d, 1))
        else
            push!(tensors, rand(D, d, d, D))
        end
    end
    return tensors
end
function createTTMPO(tensortrain; sites=nothing)
    localdims = [size(tensortrain[i])[2] for i in 1:length(tensortrain)]
    if sites === nothing
        sites = [Index(localdims[n], "n=$n") for n in 1:N]
    else
        all(localdims .== dim.(sites)) ||
            error("ranks are not consistent with dimension of sites")
    end
    linkdims = [[size(t, 1) for t in tensortrain]..., 1]
    links = [Index(linkdims[l + 1], "link,l=$l") for l in 0:N]
    tensors_ = [ITensor(deepcopy(tensortrain[n]), links[n], sites[n], sites[n]', links[n + 1])
                for n in 1:N]
    tensors_[1] *= onehot(links[1] => 1)
    tensors_[end] *= onehot(links[end] => 1)
    return MPO(tensors_)
end
"""
Create a MPS filled with zero
"""
function onemps(::Type{T}, sites::Vector{Index{Int64}}) where {T<:Number}
    M = MPS(T, sites; linkdims=1)
    l = linkinds(M)
    for n in eachindex(M)
        if n == 1
            M[n] = ITensor(T, sites[n], l[n])
        elseif n == length(M)
            M[n] = ITensor(T, l[n-1], sites[n])
        else
            M[n] = ITensor(T, l[n-1], sites[n], l[n])
        end
        M[n] .= one(T)
    end
    return M
end
#元のMPOを作成し、動かすのに使用する関数、toi.jlには含まれない。

onemps

In [11]:
function soutai(QTTres,res)
    N=length(res)
    s=[]
    for n in 1:N
        push!(s,(abs(QTTres[n]-res[n])/res[n]))
    end
    return s
end
#相対誤差算出用

soutai (generic function with 1 method)

In [2]:
include("toi.jl")
using .Toi

## 実働

### MPOの作成

In [3]:
N = 4
d = 4
D = 2
tensors_ = createTensors(N, D, d)
inds = [Index(d, "a = $i") for i in 1:N]
tt = createTTMPO(tensors_, sites=inds)
#ttとindsを今後使用。localdimは2^Rとすること

MPO
[1] ((dim=4|id=855|"a=1"), (dim=4|id=855|"a=1")', (dim=2|id=952|"l=1,link"))
[2] ((dim=2|id=952|"l=1,link"), (dim=4|id=798|"a=2"), (dim=4|id=798|"a=2")', (dim=2|id=957|"l=2,link"))
[3] ((dim=2|id=957|"l=2,link"), (dim=4|id=436|"a=3"), (dim=4|id=436|"a=3")', (dim=2|id=26|"l=3,link"))
[4] ((dim=2|id=26|"l=3,link"), (dim=4|id=941|"a=4"), (dim=4|id=941|"a=4")')


### QTTMPOの作成とvector化

QTTMPOの作成

In [5]:
cut=0.1
QTTMPO,MPOin,links,R=Toi.MPOtoQTTMPO(tt,cut)
#QTTMPO→QTT化したMPO
#MPOin→QTTMPOのindex
#links→linkindexのみをまとめたもの
#R→2^RのR
#後半の3つはMPS作成に使用
QTTMPO

MPO
[1] ((dim=2|id=751|"a=1"), (dim=2|id=220|"a=1")', (dim=4|id=395|"l=1"))
[2] ((dim=4|id=395|"l=1"), (dim=2|id=782|"a=2"), (dim=2|id=722|"a=2")', (dim=2|id=535|"l=2"))
[3] ((dim=2|id=535|"l=2"), (dim=2|id=980|"b=1"), (dim=2|id=83|"b=1")', (dim=6|id=384|"l=3"))
[4] ((dim=6|id=384|"l=3"), (dim=2|id=151|"b=2"), (dim=2|id=290|"b=2")', (dim=2|id=113|"l=4"))
[5] ((dim=2|id=113|"l=4"), (dim=2|id=907|"c=1"), (dim=2|id=825|"c=1")', (dim=6|id=405|"l=5"))
[6] ((dim=6|id=405|"l=5"), (dim=2|id=842|"c=2"), (dim=2|id=825|"c=2")', (dim=2|id=463|"l=6"))
[7] ((dim=2|id=463|"l=6"), (dim=2|id=173|"d=1"), (dim=2|id=980|"d=1")', (dim=4|id=105|"l=7"))
[8] ((dim=4|id=105|"l=7"), (dim=2|id=113|"d=2"), (dim=2|id=516|"d=2")')


QTTMPOに合わせるためのMPSを作成

In [6]:
testMPS=Toi.MPOintoMPSones(MPOin,links,R,N)
#要素は全部1

MPS
[1] ((dim=2|id=751|"a=1"), (dim=1|id=591|"l=1"))
[2] ((dim=1|id=591|"l=1"), (dim=2|id=782|"a=2"), (dim=1|id=376|"l=2"))
[3] ((dim=1|id=376|"l=2"), (dim=2|id=980|"b=1"), (dim=1|id=601|"l=3"))
[4] ((dim=1|id=601|"l=3"), (dim=2|id=151|"b=2"), (dim=1|id=823|"l=4"))
[5] ((dim=1|id=823|"l=4"), (dim=2|id=907|"c=1"), (dim=1|id=226|"l=5"))
[6] ((dim=1|id=226|"l=5"), (dim=2|id=842|"c=2"), (dim=1|id=364|"l=6"))
[7] ((dim=1|id=364|"l=6"), (dim=2|id=173|"d=1"), (dim=1|id=289|"l=7"))
[8] ((dim=1|id=289|"l=7"), (dim=2|id=113|"d=2"))


結合QTTMPOとMPSの結合→vector化

In [7]:
QTTres_= apply(QTTMPO, testMPS)
QTTresinds=Toi.restoinds(QTTres_)
QTTres = vec(Array(reduce(*, QTTres_), reverse(QTTresinds[1:Int(N*R)])))

256-element Vector{Float64}:
 121.19218713168343
 144.69787884819652
 207.2868154917067
 160.18976867188815
  89.11522809165886
 113.46706363579105
 148.8681039981855
 116.50042827434224
 104.90400364694959
 135.92541768269223
   ⋮
  95.66525618973508
  86.63309457756104
 112.23042734321331
 143.75418612309124
 112.90424669025002
  81.93020879211892
 102.56906650562699
 137.74532119565802
 107.42692391302899

### 元のMPOのvector化

In [10]:
onemps_ = onemps(Float64, inds)
res = apply(tt, onemps_)
res = vec(Array(reduce(*, res), reverse(inds[1:N])))

256-element Vector{Float64}:
 120.23450216173008
 143.5554396229856
 205.64829439549638
 158.92373576951618
  89.48961652912317
 113.94187665776317
 149.4944708816756
 116.99021025834291
 104.8486341447501
 135.82825797990034
   ⋮
  96.58252820099347
  86.8465138310228
 112.4902563718451
 144.1166951439496
 113.18542463601045
  81.68357322337202
 102.23985726545938
 137.3409462805353
 107.10726842969454

## resとQTTresの比較

In [12]:
s=soutai(QTTres,res)

256-element Vector{Any}:
 0.007965142723052532
 0.007958174404336524
 0.007967589038492939
 0.00796629211012295
 0.004183596399057816
 0.004167151146705063
 0.004189899999618543
 0.0041865211022282444
 0.0005280898759544719
 0.0007153128828778929
 ⋮
 0.009497287225174905
 0.0024574302876107963
 0.0023097914167153678
 0.002515385330591156
 0.0024842239772891665
 0.003019402298580232
 0.0032199696769220167
 0.0029443143219411508
 0.002984442494154971

In [13]:
smax=maximum(s)

0.022989488882723483

In [14]:
smin=minimum(s)

8.300375785343387e-7

In [15]:
sabr=sum(s)/length(s)

0.006499409290943023

In [16]:
println("相対誤差の最大は",smax)
println("相対誤差の最大は",smin)
println("相対誤差の平均は",sabr)

相対誤差の最大は0.022989488882723483
相対誤差の最大は8.300375785343387e-7
相対誤差の平均は0.006499409290943023
